# セマンティックセグメンテーション(PSPNet)

In [1]:
!gpustat

/bin/sh: gpustat: command not found


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [3]:
import random
import math
import time
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import transforms


# 実装したネットワークで出力
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# Setup seeds
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

## 3-1 セマンティックセグメンテーションとは
本章では画像処理タスクの１つである，セマンティックセグメンテーションに取り組みながら**PSPNet(Pyramid Scene Parsing Network)**と呼ばれるモデルについて理解する．

### セマンティックセグメンテーションの入出力
セマンティックでの入出力の関係は
- input : 画像
- output : 各ピクセルが所属するクラスのラベル情報

出力については**カラーパレット形式**よって表現し，それぞれのラベルに対して予め固有のRGBの値を付与しておきそのRGBの値を元に図示する.

### VOCデータセット
ここではVOCデータセットを用いるが，その中でもセマンティックセグメンテーション用にアノテーションが用意されている画像データのみを使用する．訓練データ1,464枚と検証データ1,449枚を用いる．

### PSPNetによる物体検出の流れ
**PSPNet(Pyramid Scene Parsing Network)** とはセマンティックセグメンテーションを行うディープラーニングアルゴリズムの一種である．

PSPNetは大きく４つのステップによって構成される．
1. 前処理を行う．具体的には画像サイズのリサイズと色情報の標準化を行う.
2. PSPNetに対して前処理した画像を入力する(H,W)．すると出力として(C+1, H, W)のサイズを持つ出力を得る．C+1は背景を加えたクラスの数を表している．この出力は各ピクセルに対するそれぞれのクラスの確信度を表す．
3. PSPNetの出力に対して，各ピクセルのクラス予測確信度が最大のものをそのピクセルのクラスと予測する．
4. 3の処理を終えた(H, W)のサイズの画像を元の画像サイズにリサイズし直して出力する．

## 3-2 DatasetとDataLoaderの実装

### 画像データ，アノテーションデータへのファイルパスのリストを作成

In [7]:
def make_datapath_list(rootpath):
    
    
    # テンプレート この%sにあとでいろいろ入れる
    imgpath_template = osp.join(rootpath, "JPEGImages", "%s.jpg")
    annopath_template = osp.join(rootpath, "SegmentationClass", "%s.png")
    
    # 訓練と検証，　それぞれのファイルIDを取得
    train_id_names = osp.join(rootpath + "ImageSets/Segmentation/train.txt")
    val_id_names = osp.join(rootpath + "ImageSets/Segmentation/val.txt")
    
    # 訓練データの画像ファイルとアノテーションファイルへのパスリスト
    train_img_list = list()
    train_anno_list = list()
    
    for line in open(train_id_names):
        file_id = line.strip()
        img_path = (imgpath_template % file_id)
        anno_path = (annopath_template % file_id)
        train_img_list.append(img_path)
        train_anno_list.append(anno_path)
        
    # 検証データの画像ファイルとアノテーションファイルへのパスリスト
    val_img_list = list()
    val_anno_list = list()
    
    for line in open(val_id_names):
        file_id = line.strip()
        img_path = (imgpath_template % file_id)
        anno_path = (annopath_template % file_id)
        val_img_list.append(img_path)
        val_anno_list.append(anno_path)
        
    return train_img_list, train_anno_list, val_img_list, val_anno_list

In [8]:
# # 動作確認
# roopath = ".data/VOCdevkit/VOC2012/"

# train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(roopath)
# print(train_img_list[0])
# print(train_anno_list[0])

### Datasetの作成
まずDataTransformを実装する．このDataTransformの詳しい内容については省略し，前処理クラスのインポートを行うことにとどめる．

1. まず対象画像とアノテーションデータをセットに対してセットで前処理を行う必要があるのでこれらをセットで変換することを可能にする```Compose```を用意する．
2. 次にこの```Compose```クラス内で種々の前処理を行うことによってモデルの汎化性能を高める．ここで行う前処理はリサイズ・切り取り・回転・色情報の標準化などがある．

セグメンテーション用のアノテーションデータには，物体の境界部分にはラベル255が特別についているのでこれは背景を表すラベル０に吸収させる．物体検出では背景に元々ラベルが与えられたなかったので全体のインデックスをインクリメントする必要があったがセグメンテーション用のアノテーションではそのようなことをする必要はない．

